In [149]:
using Feather
using LightGraphs
using GraphPlot
using Compose
using DataFrames
using Colors
using ProgressMeter
using Plots
plotly()

Plots.PlotlyBackend()

In [150]:
df = Feather.read("save.feather")
size(df, 1)

28852738

In [151]:
bias = readtable("bias.csv", header=false)
code = r"^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?"
#print(bias)
c = 0
biasnames = String[]
for i in 1:size(bias,1)
    if ~isna(bias[i, 5])
        url = match(code, AbstractString(bias[i, 5]))
        bias[i, 5] = replace(url[4], "www.", "")
        push!(biasnames, bias[i, 5])
    else
        push!(biasnames, "NA")
    end
end
bias

,x1,x2,x3,x4,x5
1,Anthropocene Magazine,C,VERY HIGH,NA,anthropocenemagazine.org
2,Raw Progressive,L,MIXED,NA,rawprogressive.com
3,The Record (NJ),LC,HIGH,NA,northjersey.com
4,The Automatic Earth,LC,HIGH,NA,theautomaticearth.com
5,Al Jazeera,LC,HIGH,NA,aljazeera.com
6,Lexington Institute,RC,HIGH,NA,lexingtoninstitute.org
7,TakePart,L,HIGH,NA,takepart.com
8,Broken World News,NA,NA,S,brokenworldnews.com
9,Townhall,R,HIGH,NA,townhall.com
10,Left Action,L,HIGH,NA,leftaction.com


In [152]:
cats = Dict{String, Dict}()
for key in ["a", "link", "script", "img"]
    cats[key] = Dict{String, Dict}()
end

range = size(df, 1)

p = Progress(range, 0.1)
for i in 1:range
    sdom = get(df[i, 1])
    ddom = get(df[i, 2])
    if isascii(sdom) && isascii(ddom)
        cat = cats[get(df[i, 3])]
        if ~haskey(cat, sdom)
            cat[sdom] = Dict{String, Int}()
        end
        if ~haskey(cat[sdom], ddom)
            cat[sdom][ddom] = 1
        else
            cat[sdom][ddom] += 1
        end
    end
    next!(p)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:01:31


In [5]:
p = Progress(size([k for k in keys(cats["a"])], 1), 0.1)
println(size([k for k in keys(cats["a"])], 1))
mut = Dict{String, Dict}()
for src in keys(cats["a"])
    for dst in keys(cats["a"][src])
        if haskey(cats["a"], dst) && haskey(cats["a"][dst], src)
            if ~haskey(mut, src)
                 mut[src] = Dict{String, Int}()
            end
            mut[src][dst] = cats["a"][src][dst]
        end
    end
    next!(p)
end
#println(mut)

18721


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


In [153]:
s2i = Dict{String, Int}()
i2s = Dict{Int, String}()
blacklist = String["twitter.com", "facebook.com", "youtube.com", "instagram.com", 
    "plus.google.com", "linkedin.com", "t.co", "itunes.apple.com", "pinterest.com", 
    "flickr.com", "bit.ly", "play.google.com"]
minsamp = 1
count = 0
body = mut
for src in keys(body)
    if ~(src in blacklist)
        for dst in keys(body[src])
            if body[src][dst] > minsamp && ~(dst in blacklist)
                if ~haskey(s2i, src)
                    count += 1
                    s2i[src] = count
                    i2s[count] = src
                end
                if ~haskey(s2i, dst)
                    count += 1
                    s2i[dst] = count
                    i2s[count] = dst
                end
            end
        end
    end
end

println("Number of node labels $(length(s2i))")

G = Graph(count)
for src in keys(body)
    if ~(src in blacklist)
        for dst in keys(body[src])
            if body[src][dst] > minsamp &&  ~(dst in blacklist)
                add_edge!(G, s2i[src], s2i[dst])
            end
        end
    end
end

println("Number of nodes $(nv(G))")
println("Number of edges $(ne(G))")


Number of node labels 4999
Number of nodes 4999
Number of edges 8707


In [154]:
sgs = sort(connected_components(G), by=size, rev=true)
println(size(sgs,1))
sg = sgs[1]
println(size(sg))

ss2i = Dict{String, Int}()
si2s = Dict{Int, String}()

c = 0
for i in sg
    c += 1
    ss2i[i2s[i]] = c
    si2s[c] = i2s[i]
end

H = Graph(size(sg, 1))

for j in 1:c
    src = si2s[j]
    if src in keys(body)
        for dst in keys(body[src])
            if body[src][dst] > minsamp && ~(dst in blacklist)
                add_edge!(H, j, ss2i[dst])
            end
        end
    end
end
#lx, ly = spring_layout(H)

1770
(1643,)


In [155]:

sizes = [0.2 + log(length(all_neighbors(H, v))) for v in 1:c]
labels = [si2s[v] for v in 1:c]

function reality(x, key)
    if x in biasnames
        v = bias[find(biasnames .== x)[1], 4]
        if key == "fake"
            if isna(v)
                return 2
            else
                return 3
            end
        elseif key == "bias"
            if isna(bias[find(biasnames .== x)[1], 2])
                return 1
            elseif (bias[find(biasnames .== x)[1], 2] in ["L", "LC"])
                return 2
            elseif (bias[find(biasnames .== x)[1], 2] in ["R", "RC"])
                return 3
            end
        end
    end
    return 1
end

members = [reality(si2s[v], "fake") for v in 1:c]
colors = [colorant"grey", colorant"blue", colorant"red"]
#draw(PNG("plots/beliefprop.png", 60cm, 60cm),
#     gplot(H, lx, ly, nodelabel=labels, nodelabelsize=sizes, nodefillc=colors[members]))


In [156]:
using Iterators

function normalize_neighbors!(m, g::Graph, X::AbstractArray, i::Integer)
    Z = sum(m[i,neighbors(g,i), :])
    #@show i, Z
    for j in neighbors(g,i)
        for k in X
            m[i,j,k] /= Z
        end
    end
    return Z
end

function assert_nonzero(val)
    @assert val != 0 "val was zero"
    return val
end

function propogate!{T<:Real, U<:Integer}(m::AbstractArray{T,3},
                                         ϕ::AbstractArray{T,2},
                                         ψ::AbstractArray{T,2}, A, X::AbstractArray{U, 1})
    n = size(A, 1)
    for i in 1:n
        for j in neighbors(A,i)
            if j == i
                continue
            end
            for k in X
                m[i,j,k] = 0
                for l in X
                    w = ϕ[i,l] * ψ[l,k]
                    @assert w != NaN
                    sp = sum([p!=j ? log(assert_nonzero(m[p,i,l])) : 0 for p in neighbors(A, i)])
                    p = exp(sp) + 1e-16
                    if p == 0
                        warn("sp is $sp")
                        warn("p is 0! $i,$j,$k,$l")
                    end
                    @assert p != NaN
                    @assert m[j,i,l] != 0 "$i,$j,$l have m 0"
                    m[i,j,k] += w * p #/ m[j,i,l]
                end
            end
        end
        Z = normalize_neighbors!(m, A, X, i)
    end
    return m
end

function beliefs!{T<:Real}(m::AbstractArray{T}, b, ϕ, A, X)
    for i in 1:size(A,1)
        for k in X
            w = prod(T[m[p,i,k] for p in neighbors(A, i)])
            if w == 0
                w = 1e-16
            end
            b[i,k] = ϕ[i,k] * w
            @assert b[i,k] >= 0 "Unnormalized beliefs are not all positive offending indices ($i, $k)=$(b[i,k])"
        end
        b[i,:] /= sum(b[i,:])
    end
    #@show b
    @assert all( 0.99 .< sum(b,2) .< 1.01) "Beliefs do not form a probability distributions"
    return b
end

type BeliefProblem{T<:Real, U<:Integer, G}
    m::AbstractArray{T,3}
    ϕ::AbstractArray{T,2}
    ψ::AbstractArray{T,2}
    A::G
    X::AbstractArray{U, 1}
end

type Messages{T<:Real, G} <: AbstractArray{T, 3}
    nrows::Int
    ncols::Int
    edges::Dict{Tuple{Int, Int}, Int}
    states::UnitRange
    storage::Array{T, 2}
end

function Messages(g::Graph, states::UnitRange)
    n=nv(g)
    d = Dict{Tuple{Int,Int}, Int}()
    i = 1
    for e in edges(g)
        d[(src(e), dst(e))] = i
        d[(dst(e), src(e))] = i+1
        i += 2
    end
    storage = ones(Float64, (i, length(states)))
    return Messages{Float64, Graph}(n,n,d, states, storage)
end

Base.size(m::Messages) = (m.nrows, m.ncols, length(m.states))
function Base.getindex(m::Messages, I::Vararg{Int})
    pair = I[1:2]
    if pair in keys(m.edges)
        return m.storage[m.edges[pair], I[3]]
    else
        throw(KeyError(pair))
    end
end

function Base.setindex!(m::Messages, v, I::Vararg{Int})
    pair = I[1:2]
    if pair in keys(m.edges)
        return m.storage[m.edges[pair], I[3]] = v
    else
        throw(KeyError(pair))
    end
end

logodds(b::AbstractMatrix) = log.(b[:,1]./b[:,2])

function beliefprop(g::Graph, ϕ, ψ, maxiter)
    n = size(g, 1)
    k = 2
    m = ones(Float64, (n,n,k))
    m = Messages(g, 1:k)
    #@show sort(collect(keys(m.edges)))
    pr = BeliefProblem(m, ϕ, ψ, g, 1:k)
    iterstates = []
    diffs = []
    i = 1
    m = nth(iterate(m->begin msg = propogate!(m, ϕ,ψ, g, 1:k);
            state = [maximum(msg.storage), minimum(msg.storage)]
            diff = i > 1 ? state-iterstates[i-1]:0;
            i+=1
            #@show diff
            push!(iterstates, state)
            push!(diffs, diff)
            #msg.storage = max(msg.storage, 1e-60)
           return msg 
        end, m), maxiter)
    b = similar(pr.ϕ)
#     for i in 1:n
#         if sum(pr.ϕ[i,:]) != sum(b[i, :])
#             println("Crud! "*string(pr.ϕ[i,:])*" "*string(b[i,:]))
#         end
#     end
    beliefs!(pr.m, b, pr.ϕ, g, 1:k)
    lodds = logodds(b)
    plotbeliefs(g, b[:,1])
    #display(bar(lodds, ylabel="Log Odds", xlabel="Vertex ID"))
    return pr, b, lodds
end

function plotbeliefs(g::Graph, b::AbstractVector)
    nodefillc=map(x-> RGB(1x, 0, 1-x), b./maximum(b))
    #plo = gplot(H, lx, ly, nodelabel=[si2s[v] for v in 1:nv(g)], nodelabelsize=sizes, nodefillc=nodefillc)
    #draw(PNG("plots/postbeliefprop.png", 60cm, 60cm), plo)
    #return plo
    #return gplot(g, layout=(g)-> spring_layout(g; C=1), nodelabel=1:nv(g), nodelabelsize=1, nodefillc=nodefillc)
    #return gplot(g, layout=(g)-> begin pos = (200*begin Z=spring_layout(g); Z[1]end, log2(float(collect(1:nv(g))))); @show typeof(pos); return pos end, nodelabel=1:nv(g), nodelabelsize=1, nodefillc=nodefillc)
    #return gplot(g, layout=(g)-> (exp10(spectral_layout(g)[1]), log(float(vertices(g)))),nodelabel=1:nv(g), nodelabelsize=1, nodefillc=nodefillc)
end

function Psis(ϵ::Real)
    return [1-ϵ ϵ;
            ϵ 1-ϵ]
end

Psis (generic function with 1 method)

In [157]:
function AUC(xs, ys)
    ps = Array{Float64}(size(xs, 1), 2)
    ps[:, 1] = xs
    ps[:, 2] = ys
    ps = sortrows(ps, by=x->(x[1],x[2]))
    s = 0
    
    for i in 1:size(ps, 1)-1
        s += (ps[i+1,1]-ps[i,1])*(ps[i,2]+ps[i+1,2])/2
    end
    return s
end

AUC (generic function with 2 methods)

In [118]:
# Learning Curve
srand(42)
v2r = Dict{Int, Int}()
folds = Dict{Int, Int}()
order = Dict{Int, Int}()
rando = randn(c)
i = 0
for v in sort(sortperm(rando))
    order[v] = i
    i += 1
    folds[v] = rand(1:fc)
    v2r[v] = reality(si2s[v], "fake")
end

#plt = plot()

fc = 3
sizes = []
scores = []
for size in linspace(10,c,10)
    
    order = Dict{Int, Int}()
    rando = randn(c)
    i = 0
    for v in sort(sortperm(rando))
        order[v] = i
        i += 1
    end
    
    trainsize = 0
    testsize = 0
    ϕ = ones((c,2))./2.0
    for v in 1:c
        if order[v] < size
            r = v2r[v]
            if r == 3
                trainsize += 1
                ϕ[v,:] = [0.99, 0.01]
            elseif r == 2
                trainsize += 1
                ϕ[v,:] = [0.01, 0.99]
            end
        else
            r = v2r[v]
            if r != 1
                testsize += 1
            end
        end
    end
    println("Train Size $trainsize")
    println("Test Size $testsize")
    pr, b, lodds = beliefprop(H, ϕ, Psis(0.325), 10);

    roc_x = []
    roc_y = []
    for cutoff in linspace(0, 1, 51)

        score = zeros(Int, (2, 2))
        for v in 1:c
            if order[v] >= size
                r = v2r[v]
                if r == 3
                    if b[v, 1] > cutoff
                        score[1,1] += 1
                    else
                        score[2,1] += 1
                    end
                elseif r == 2
                    if b[v, 1] > cutoff
                        score[1,2] += 1
                    else
                        score[2,2] += 1
                    end
                end
            end
        end
        tp = score[1,1]/sum(score[:,1])
        fp = score[1,2]/sum(score[:,2])
        append!(roc_x, fp)
        append!(roc_y, tp)
    end
    println(AUC(roc_x, roc_y))
    append!(scores, AUC(roc_x, roc_y))
    append!(sizes, trainsize)
end
plot(sizes, scores)

Train Size 7
Test Size 377
0.7165958451369216
Train Size 74
Test Size 310
0.7829982517482518
Train Size 134
Test Size 250
0.7939092127944838
Train Size 177
Test Size 207
0.8658476658476657
Train Size 207
Test Size 177
0.8881257631257632
Train Size 249
Test Size 135
0.9044918330308529
Train Size 288
Test Size 96
0.8679012345679012
Train Size 325
Test Size 59
0.7232704402515724
Train Size 351
Test Size 33
1.0
Train Size 384
Test Size 0
NaN


In [162]:
srand(42)
fc = 3
folds = Dict{Int, Int}()
v2r = Dict{Int, Int}()
for v in 1:c
    folds[v] = rand(1:fc)
    v2r[v] = reality(si2s[v], "fake")
end

scores = zeros(Float64, 3, 5)
println(scores)
println(typeof(scores))

for (x, its) in enumerate(linspace(10, 14, 3))
    println("Its $its")
    for (y, psi) in enumerate(linspace(0.375, 0.475, 5))
        println("Psi $psi")
        for f in 1:fc
            ϕ = ones((c,2))./2.0
            for v in 1:c
                if ~(folds[v] == f)
                    r = v2r[v]
                    if r == 3
                        ϕ[v,:] = [0.99, 0.01]
                    elseif r == 2
                        ϕ[v,:] = [0.01, 0.99]
                    end
                end
            end
            pr, b, lodds = beliefprop(H, ϕ, Psis(psi), round(Int, its));

            roc_x = []
            roc_y = []
            for cutoff in linspace(0, 1, 51)
                score = zeros(Int, (2, 2))
                for v in 1:c
                    r = v2r[v]
                    if r == 3
                        if b[v, 1] > cutoff
                            score[1,1] += 1
                        else
                            score[2,1] += 1
                        end
                    elseif r == 2
                        if b[v, 1] > cutoff
                            score[1,2] += 1
                        else
                            score[2,2] += 1
                        end
                    end
                end
                tp = score[1,1]/sum(score[:,1])
                fp = score[1,2]/sum(score[:,2])
                append!(roc_x, fp)
                append!(roc_y, tp)
            end
            acc = AUC(roc_x, roc_y)
            scores[x, y] += acc/3
        end
        println(scores)
    end
end

println(scores)

[0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]
Array{Float64,2}
Its 10.0
Psi 0.375
[0.981752 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]
Psi 0.4
[0.981752 0.981867 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]
Psi 0.425
[0.981752 0.981867 0.982002 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]
Psi 0.45
[0.981752 0.981867 0.982002 0.981848 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]
Psi 0.475
[0.981752 0.981867 0.982002 0.981848 0.981983; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]
Its 12.0
Psi 0.375
[0.981752 0.981867 0.982002 0.981848 0.981983; 0.981752 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]
Psi 0.4
[0.981752 0.981867 0.982002 0.981848 0.981983; 0.981752 0.981867 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]
Psi 0.425
[0.981752 0.981867 0.982002 0.981848 0.981983; 0.981752 0.981867 0.982002 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]
Psi 0.45
[0.981752 0.981867 0.982002 0.981848 0.981983; 0.981752 0.981867 0.982002 0.981848 0.0; 0.0 0.0 0.0 0.0 0.0]
Psi 0.475
[0.981

In [158]:
roc_cuts = Dict{Float64, Dict}()
roc_x = Dict{Float64, Array}()
roc_y = Dict{Float64, Array}()
srand(42)
fc = 3

folds = Dict{Int, Int}()
v2r = Dict{Int, Int}()
for v in 1:c
    folds[v] = rand(1:fc)
    v2r[v] = reality(si2s[v], "bias")
end

for psi in linspace(0, 0.5, 41)
    println("Psi $psi")
    roc_cuts[psi] = Dict{Float64, Array}()

    for f in 1:fc
        println("Fold $f")

        ϕ = ones((c,2))./2.0
        for v in 1:c
            if folds[v] != f 
                r = v2r[v]
                if r == 3
                    ϕ[v,:] = [0.99, 0.01]
                elseif r == 2
                    ϕ[v,:] = [0.01, 0.99]
                end 
            end
        end
        pr, b, lodds = beliefprop(H, ϕ, Psis(psi), 2);

        
        for lvl in linspace(0, 1, 50)
            if ~haskey(roc_cuts[psi], lvl)
                roc_cuts[psi][lvl] = []
            end
            cutoff = lvl
            #println("Cutoff is $cutoff")

            score = zeros(Int, (2, 2))

            for v in 1:c
                if folds[v] == f
                    r = v2r[v]
                    if r == 3
                        if b[v, 1] > cutoff
                            score[1,1] += 1
                        else
                            score[2,1] += 1
                        end
                    elseif r == 2
                        if b[v, 1] > cutoff
                            score[1,2] += 1
                        else
                            score[2,2] += 1
                        end
                    end
                end
            end
            #println("Score : "*string(100*(score[1,1]+score[2,2])/sum(score))*"% "*
            #        "Recovered "*string(score[1,1]+score[2,2])*" of "*string(sum(score)))
            #println(score)
            tp = score[1,1]/sum(score[:,1])
            fp = score[1,2]/sum(score[:,2])
            push!(roc_cuts[psi][lvl],(tp,fp))
        end
    end
    
    roc_ps = []
    for key in keys(roc_cuts[psi])
        push!(roc_ps, (sum([x[1] for x in roc_cuts[psi][key]])/fc, sum([x[2] for x in roc_cuts[psi][key]])/fc))
    end
    roc_line = sort(roc_ps, by=x->x[2]) 
    roc_x[psi] = [p[2] for p in roc_line]
    roc_y[psi] = [p[1] for p in roc_line]
end


Psi 0.0
Fold 1
Fold 2
Fold 3
Psi 0.0125
Fold 1
Fold 2
Fold 3
Psi 0.025
Fold 1
Fold 2
Fold 3
Psi 0.0375
Fold 1
Fold 2
Fold 3
Psi 0.05
Fold 1
Fold 2
Fold 3
Psi 0.0625
Fold 1
Fold 2
Fold 3
Psi 0.075
Fold 1
Fold 2
Fold 3
Psi 0.0875
Fold 1
Fold 2
Fold 3
Psi 0.1
Fold 1
Fold 2
Fold 3
Psi 0.1125
Fold 1
Fold 2
Fold 3
Psi 0.125
Fold 1
Fold 2
Fold 3
Psi 0.1375
Fold 1
Fold 2
Fold 3
Psi 0.15
Fold 1
Fold 2
Fold 3
Psi 0.1625
Fold 1
Fold 2
Fold 3
Psi 0.175
Fold 1
Fold 2
Fold 3
Psi 0.1875
Fold 1
Fold 2
Fold 3
Psi 0.2
Fold 1
Fold 2
Fold 3
Psi 0.2125
Fold 1
Fold 2
Fold 3
Psi 0.225
Fold 1
Fold 2
Fold 3
Psi 0.2375
Fold 1
Fold 2
Fold 3
Psi 0.25
Fold 1
Fold 2
Fold 3
Psi 0.2625
Fold 1
Fold 2
Fold 3
Psi 0.275
Fold 1
Fold 2
Fold 3
Psi 0.2875
Fold 1
Fold 2
Fold 3
Psi 0.3
Fold 1
Fold 2
Fold 3
Psi 0.3125
Fold 1
Fold 2
Fold 3
Psi 0.325
Fold 1
Fold 2
Fold 3
Psi 0.3375
Fold 1
Fold 2
Fold 3
Psi 0.35
Fold 1
Fold 2
Fold 3
Psi 0.3625
Fold 1
Fold 2
Fold 3
Psi 0.375
Fold 1
Fold 2
Fold 3
Psi 0.3875
Fold 1
Fold 2
Fold 3
Psi 

In [159]:
plt = plot()
for f in linspace(0, 0.5, 41)
    #println(hex(round(Int,400*f),2))
    plot!(plt, roc_x[f], roc_y[f], lab=string(f), linecolor="#"*hex(round(Int,400*f),2)*"0000")
end
plot!(plt, [0, 1], [0, 1], lab="random", style=:dash)
plt

In [160]:
plt = plot()
auc_x = []
auc_y = []
for f in linspace(0, 0.5, 21)
    append!(auc_x, f)
    append!(auc_y, AUC(roc_x[f], roc_y[f]))
end
plot!(plt, auc_x, auc_y)
plt